In [ ]:
20

In [ ]:
20

In [1]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model




### Register model

In [2]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [3]:

from federatedscope.contrib.trainer.laplacian_trainer_with_domain_separation_VAE_separated_other_diff_other_sim import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "


Device: cuda


### Register metric

### Create the config

#### Set data, model, trainer and metric

In [4]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("scripts/B-FHTL_exp_scripts/Graph-DC/fedDomSep_other_diff_other_sim.yaml")
cfg.data.save_dir = 'test_dir'
# cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
cfg.trainer.type = 'laplacian_trainer'
cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']
cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.csd_importance= 0
cfg.params.sim_importance = 1.
cfg.params.diff_importance = 0.01
cfg.params.eps=1e-20
cfg.params.p=0.
cfg.params.lam = 0.
cfg.params.recon_importance = 0.01
cfg.params.kld_importance = 0.
cfg.model.dropout=0.5


#### configure other options

In [5]:
#cfg.use_gpu = True 0.05 [0.02-0.03] 0.04-0.06
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'

cfg.federate.method = \
    f'Laplacian_MINE_VAE_test_no_KLD'

#cfg.federate.local_update_steps = 20000000
cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 2000
cfg.federate.client_num = 13
cfg.early_stop.patience = 3000
#cfg.train.optimizer.lr = 0.001
#cfg.train.optimizer.weight_decay = 0.0005
#cfg.grad.grad_clip = 2.0
cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'


In [6]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(F/home/michael/Desktop/backup files/FederatedScope/data/CIKM22Competitionalse)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [7]:
from yacs.config import CfgNode
client_cfg_file = "scripts/B-FHTL_exp_scripts/Graph-DC/cfg_per_client.yaml"# "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_ours_lr_local_steps.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [8]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


2023-01-16 17:16:22,203 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [9]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_MI_client import LaplacianDomainSeparationMIClient
from federatedscope.contrib.workers.laplacian_server_dom_sep import LaplacianServerDomSep

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2023-01-16 17:16:22,216 (utils:129)INFO: the current machine is at 127.0.1.1
2023-01-16 17:16:22,217 (utils:131)INFO: the current dir is /home/michael/Master-Thesis/CKIM_Competition
2023-01-16 17:16:22,218 (utils:132)INFO: the output dir is exp/test_dir/Laplacian_MINE_VAE_test_no_KLD_gin_on_fs_contest_data_lr0.5_lstep1_/sub_exp_20230116171622


In [10]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_VAE_Separated_otherDiff_otherSim_client import LaplacianDomainSeparation1MINE_Separated_OtherDiff_OtherSim_Client
from federatedscope.contrib.workers.laplacian_with_domain_separation_1MINE_Other_DiffLoss_VAE_client import LaplacianDomainSeparation1MINEOtherDiffClient
from federatedscope.contrib.workers.laplacian_server import LaplacianServer

Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianDomainSeparation1MINE_Separated_OtherDiff_OtherSim_Client,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

rho: 0.0
server params: 
encoder_atom.atom_embedding_list.0.weight


2023-01-16 17:16:32,429 (fed_runner:249)INFO: Server #0 has been set up ... 
2023-01-16 17:16:32,444 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 13
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/
  save_dir: test_dir
  server_holds_all: False
  shuffle: True
  sizes: [10, 5]
  splits: [0.8, 0.1, 0.1]
  splitter: ooxx
  splitter_args: []
  subsample: 1.0
  target_transform: [

encoder_atom.atom_embedding_list.1.weight
encoder_atom.atom_embedding_list.2.weight
encoder_atom.atom_embedding_list.3.weight
encoder_atom.atom_embedding_list.4.weight
encoder_atom.atom_embedding_list.5.weight
encoder_atom.atom_embedding_list.6.weight
encoder.weight
encoder.bias
local_gnn.convs.0.nn.linears.0.weight
local_gnn.convs.0.nn.linears.0.bias
local_gnn.convs.0.nn.linears.1.weight
local_gnn.convs.0.nn.linears.1.bias
local_gnn.convs.0.nn.norms.0.weight
local_gnn.convs.0.nn.norms.0.bias
local_gnn.convs.0.nn.norms.1.weight
local_gnn.convs.0.nn.norms.1.bias
local_gnn.convs.1.nn.linears.0.weight
local_gnn.convs.1.nn.linears.0.bias
local_gnn.convs.1.nn.linears.1.weight
local_gnn.convs.1.nn.linears.1.bias
local_gnn.convs.1.nn.norms.0.weight
local_gnn.convs.1.nn.norms.0.bias
local_gnn.convs.1.nn.norms.1.weight
local_gnn.convs.1.nn.norms.1.bias
global_gnn.convs.0.nn.linears.0.weight
global_gnn.convs.0.nn.linears.0.bias
global_gnn.convs.0.nn.linears.1.weight
global_gnn.convs.0.nn.linears

2023-01-16 17:16:32,621 (trainer:50)WARNING: Will not use monitor in trainer with class <class 'federatedscope.contrib.trainer.laplacian_trainer_with_domain_separation_VAE_separated_other_diff_other_sim.LaplacianDomainSeparationVAE_Separated_OtherDiff_OtherSim_Trainer'>
2023-01-16 17:16:32,622 (fed_runner:302)INFO: Client 4 has been set up ... 
2023-01-16 17:16:32,631 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 13
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_t

rho: 0.0
rho: 0.0
rho: 0.0
rho: 0.0
rho: 0.0
rho: 0.0


...

rho: 0.0


2023-01-16 17:16:33,052 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 13
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/
  save_dir: test_dir
  server_holds_all: False
  shuffle: True
  sizes: [10, 5]
  splits: [0.8, 0.1, 0.1]
  splitter: ooxx
  splitter_args: []
  subsample: 1.0
  target_transform: []
  transform: []
  type: fs_contest_data
device: 0
distribute:
  use: False


rho: 0.0
rho: 0.0


2023-01-16 17:16:39,121 (laplacian_with_domain_separation_VAE_Separated_otherDiff_otherSim_client:142)INFO: {'Role': 'Client #7', 'Round': 0, 'Results_raw': {'train_loss': 1666.736176, 'train_avg_loss': 0.675887, 'train_total': 2466, 'train_imp_ratio': 7.831862}}
2023-01-16 17:16:40,384 (laplacian_with_domain_separation_VAE_Separated_otherDiff_otherSim_client:142)INFO: {'Role': 'Client #12', 'Round': 0, 'Results_raw': {'train_loss': 115.558081, 'train_avg_loss': 0.192597, 'train_total': 600, 'train_imp_ratio': -2.368866}}
2023-01-16 17:16:40,963 (laplacian_with_domain_separation_VAE_Separated_otherDiff_otherSim_client:142)INFO: {'Role': 'Client #5', 'Round': 0, 'Results_raw': {'train_loss': 146.660852, 'train_avg_loss': 0.711946, 'train_total': 206, 'train_imp_ratio': -6.681263}}
2023-01-16 17:16:41,678 (laplacian_with_domain_separation_VAE_Separated_otherDiff_otherSim_client:142)INFO: {'Role': 'Client #11', 'Round': 0, 'Results_raw': {'train_loss': 149.665855, 'train_avg_loss': 0.7126

KeyboardInterrupt: 

client 0 times: 0.346s
bei normal fedavg: 0.15s
mit MINE: 1.7 Sekunden


+

In [1]:
777-431

346

In [ ]:
data

In [ ]:
import os

,,0After 10 rounds:

diff: 0, csd: 0:
server: -8
c1 t/val: 0.375 /0.37
c2 t/val: 0.678 /0.606
c3 t/val: 0.699 /0.608
c4 t/val: 0.599 /0.537

diff: 0, csd: 0:
server: -4.5
c1 t/val: 0.372879 /0.328
c2 t/val: 0.678992 /0.605809
c3 t/val: 0.70166 /0.608609
c4 t/val: 0.597455 /0.534253



diff: 10, csd: 0:
server: -35
c1 t/val: 0.482/0.52
c2 t/val: 0.694/0.65
c3 t/val: 0.724/0.65
c4 t/val: 0.674/0.575

diff: 1, csd: 0:
server: -9.6
c1 t/val: 0.378194/0.319349
c2 t/val: 0.67461/0.609163
c3 t/val: 0.705559/0.615019
c4 t/val: 0.605581/0.572086

diff: 0, csd: 1e4:
server: -18
c1 t/val: 0.280962/0.23074
c2 t/val: 0.682652/0.603175
c3 t/val: 0.699094/0.67027
c4 t/val: 0.594985/0.609611

diff: 0 csd: 1e8
server: -18
c1 t/val: 0.2606/0.348(0.197)
c2 t/val: 0.653/0.0.602
c3 t/val: 0.691/0.597
c4 t/val: 0.581/0.629

diff: 0 csd: 1e8
server: -12.7
c1 t/val: 0.262647/0.249875
c2 t/val: 0.649273/0.0.600435
c3 t/val: 0.696132/0.615158
c4 t/val: 0.586597/0.656992


diff: 0.1 csd: 0
server: -11
c1 t/val: 0.2606/0.332837
c2 t/val: 0.678326/0.0.610099
c3 t/val: 0.704083/0.616179
c4 t/val: 0.604177/0.559658




In [ ]:
os.environ['CUBLAS_WORKSPACE_CONFIG']